In [13]:
# enable intellisense
%config IPCompleter.greedy=True

In [14]:
import xml.etree.ElementTree as eTree

In [9]:
tree = eTree.parse(r'C:\Users\chanm\cse_160\project\data\interpersonal.stackexchange.com\Posts.xml')

In [10]:
root = tree.getroot()

In [12]:
root.tag


{}